In [1]:
#%% Imports
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

# Import "local" packages
import hyspec_io, hyspec_ml, image_render

In [2]:
image_base_dir = Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/Hyperspectral/20230621/2a_Rrs')
image_output_dir = Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/Hyperspectral/20230621/3a_Rrs_SGC')
pca_output_dir = image_output_dir / 'PCA_SGC_Models'
image_output_dir.mkdir(exist_ok=True)
pca_output_dir.mkdir(exist_ok=True)

In [3]:
# Define image groups (images taken during same flight are in same group)
filename_bases = ('Maholmen_02_Pika_L_',)*4 + ('Maholmen_03_Pika_L_',)*2
file_number_ranges = ((1,20), (21,44), (45,66), (67,75), (1,28), (29,40))
filename_endings = ('-Crop Wavelengths.bip.hdr',)*6
sgc_training_file_numbers = ((1,6,8,11,15,18),(25,27,32,38,43),(48,52,57,65),(68,73,75),(2,6,15,25),(30,32,39))
#sgc_training_file_numbers = sgc_training_file_numbers[0:2] # Limit for testing 


In [4]:
for filename_base, file_number_range, filename_end, train_file_nums in zip(
    filename_bases, file_number_ranges, filename_endings, sgc_training_file_numbers):

    # Define file names
    image_file_names = [filename_base + str(i) + filename_end for i in range(file_number_range[0],file_number_range[1]+1)]
    image_file_paths = [image_base_dir / im_file for im_file in image_file_names]
    training_files = [filename_base + str(i) + filename_end for i in train_file_nums]
    training_file_paths = [image_base_dir / im_file for im_file in training_files]
    
    # Train PCA model
    W_pca, X_mean = hyspec_ml.train_image_pca(image_paths=training_file_paths,random_sample_frac=0.1,
                                              verbose=True,normalize=False)

    # Save PCA model
    pca_file_name = f'{filename_base}PCA-SGC_{file_number_range[0]:02d}-{file_number_range[1]:02d}.npz'
    pca_file_path = pca_output_dir / pca_file_name
    np.savez(pca_file_path,
        W_pca = W_pca,
        X_mean = X_mean)
    
    # Apply PCA-SGC to all images and save corrected images:
    for i,image_file_path in enumerate(image_file_paths):
        print(f'Processing file {image_file_path.name}, number {i+1} of {len(image_file_paths)} in group')
        (image,wl,rgb_ind,metadata) = hyspec_io.load_envi_image(image_file_path)
        image_noglint = hyspec_ml.remove_sun_glint_pca(image,W_pca,X_mean)

        sgc_file_base, sgc_file_ext = image_file_path.name.split('.',maxsplit=1)
        hyspec_ml.save_pca_scg_image_envi(image_noglint,metadata,image_output_dir,
                                          sgc_file_base,header_file_ext=sgc_file_ext)

Opening image 1 of 6
Opening image 2 of 6
Opening image 3 of 6
Opening image 4 of 6
Opening image 5 of 6
Opening image 6 of 6
Collected data matrix has shape (1070935, 255)
Fitting PCA model...
Done.
Processing file Maholmen_02_Pika_L_1-Crop Wavelengths.bip.hdr, number 1 of 20 in group
Processing file Maholmen_02_Pika_L_2-Crop Wavelengths.bip.hdr, number 2 of 20 in group
Processing file Maholmen_02_Pika_L_3-Crop Wavelengths.bip.hdr, number 3 of 20 in group
Processing file Maholmen_02_Pika_L_4-Crop Wavelengths.bip.hdr, number 4 of 20 in group
Processing file Maholmen_02_Pika_L_5-Crop Wavelengths.bip.hdr, number 5 of 20 in group
Processing file Maholmen_02_Pika_L_6-Crop Wavelengths.bip.hdr, number 6 of 20 in group
Processing file Maholmen_02_Pika_L_7-Crop Wavelengths.bip.hdr, number 7 of 20 in group
Processing file Maholmen_02_Pika_L_8-Crop Wavelengths.bip.hdr, number 8 of 20 in group
Processing file Maholmen_02_Pika_L_9-Crop Wavelengths.bip.hdr, number 9 of 20 in group
Processing file M